## 介绍

本课将涵盖：
- 什么是函数调用及其应用场景
- 如何使用 OpenAI 创建函数调用
- 如何将函数调用集成到应用程序中

## 学习目标

完成本课后，你将了解并掌握：

- 使用函数调用的目的
- 使用 OpenAI 服务设置函数调用
- 针对你的应用场景设计高效的函数调用


## 理解函数调用

在本课中，我们要为我们的教育初创公司开发一个功能，让用户可以通过聊天机器人查找技术课程。我们会根据用户的技能水平、当前职位和感兴趣的技术推荐合适的课程。

为实现这个目标，我们将结合使用以下内容：
 - `OpenAI` 为用户创建聊天体验
 - `Microsoft Learn Catalog API` 根据用户的请求帮助查找课程
 - `Function Calling` 将用户的查询发送到函数，从而发起 API 请求

首先，让我们看看为什么要使用函数调用：

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # 获取 GPT 的新回复，此时它可以看到函数的响应

print(second_response.choices[0].message)


### 为什么要使用函数调用

如果你已经完成了本课程的其他课程，你可能已经了解了使用大型语言模型（LLM）的强大之处。希望你也能看到它们的一些局限性。

函数调用是 OpenAI 服务中的一个功能，旨在解决以下问题：

响应格式不一致：
- 在函数调用出现之前，大型语言模型的响应通常是无结构且不一致的。开发者不得不编写复杂的校验代码来处理输出中的各种变化。

与外部数据集成受限：
- 在有了这个功能之前，很难将应用程序其他部分的数据整合到聊天上下文中。

通过标准化响应格式并实现与外部数据的无缝集成，函数调用简化了开发流程，减少了额外校验逻辑的需求。

用户无法获得类似“斯德哥尔摩现在的天气如何？”这样的答案。这是因为模型只能访问其训练时的数据。

让我们来看下面的例子，来说明这个问题：

假设我们想创建一个学生数据库，以便为他们推荐合适的课程。下面有两个学生的描述，他们的数据内容非常相似。


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

我们希望将此内容发送给一个大语言模型（LLM）来解析数据。之后可以在我们的应用程序中将其发送到 API 或存储到数据库中。

让我们创建两个完全相同的提示，来指示 LLM 我们感兴趣的信息：


我们想把这个发送给一个大型语言模型，以解析对我们的产品重要的部分。这样我们可以创建两个相同的提示来指示大型语言模型：


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


创建这两个提示后，我们将使用 `openai.ChatCompletion` 将它们发送给 LLM。我们将提示存储在 `messages` 变量中，并将角色分配为 `user`。这样做是为了模拟用户向聊天机器人发送消息。


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

现在我们可以将这两个请求发送给LLM，并检查我们收到的响应。


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

即使提示是一样的，描述也很相似，我们得到的 `Grades` 属性格式也可能不同。

如果你多次运行上面的单元格，格式可能是 `3.7` 或 `3.7 GPA`。

这是因为大语言模型（LLM）接收的是非结构化的文本提示，返回的也是非结构化的数据。我们需要一个结构化的格式，这样在存储或使用这些数据时才知道应该期待什么。

通过使用函数调用，我们可以确保收到的是结构化的数据。在使用函数调用时，LLM 实际上并不会真正调用或运行任何函数。相反，我们为 LLM 创建一个结构，让它按照这个结构来生成回复。然后我们可以利用这些结构化的回复，来决定在应用程序中应该运行哪个函数。


![函数调用流程图](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.zh.png)


### 使用函数调用的用例

**调用外部工具**  
聊天机器人非常擅长为用户提供问题的答案。通过使用函数调用，聊天机器人可以根据用户的消息完成特定任务。例如，学生可以让聊天机器人“给我的老师发封邮件，说我在这个科目上需要更多帮助”。这时可以调用 `send_email(to: string, body: string)` 这个函数。

**创建 API 或数据库查询**  
用户可以用自然语言查找信息，系统会将其转换为格式化的查询或 API 请求。例如，老师可以询问“哪些学生完成了上一次作业”，这可以调用名为 `get_completed(student_name: string, assignment: int, current_status: string)` 的函数。

**生成结构化数据**  
用户可以将一段文本或 CSV，通过大语言模型提取出重要信息。例如，学生可以把一篇关于和平协议的维基百科文章转换成 AI 闪卡。这可以通过调用 `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` 这个函数来实现。


## 2. 创建你的第一个函数调用

创建函数调用的过程包括三个主要步骤：
1. 使用你的函数列表和用户消息调用 Chat Completions API
2. 读取模型的响应以执行某个操作，比如执行一个函数或 API 调用
3. 使用你的函数返回的结果，再次调用 Chat Completions API，用这些信息生成对用户的回复


![函数调用的流程](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.zh.png)


### 函数调用的要素

#### 用户输入

第一步是创建一条用户消息。你可以通过获取文本输入的值来动态赋值，也可以直接在这里指定一个值。如果你是第一次使用 Chat Completions API，我们需要定义消息的 `role` 和 `content`。

`role` 可以是 `system`（设定规则）、`assistant`（模型）或 `user`（终端用户）。在函数调用中，我们会将其设为 `user`，并给出一个示例问题。


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### 创建函数

接下来我们将定义一个函数以及它的参数。这里我们只会用一个叫做 `search_courses` 的函数，但你也可以创建多个函数。

**重要提示**：函数会被包含在系统消息中发送给 LLM，并且会占用你可用的 token 数量。


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**定义**

函数定义结构有多个层级，每一层都有自己的属性。下面是嵌套结构的详细说明：

**顶层函数属性：**

`name` -  要调用的函数名称。

`description` - 这是对函数工作方式的描述。在这里，描述要具体、清晰。

`parameters` - 你希望模型在响应中生成的值和格式的列表。

**参数对象属性：**

`type` - 参数对象的数据类型（通常为 "object"）

`properties` - 模型在响应中会用到的具体值的列表

**单个参数属性：**

`name` - 由属性键隐式定义（例如 "role"、"product"、"level"）

`type` - 该参数的数据类型（例如 "string"、"number"、"boolean"）

`description` - 对该参数的描述

**可选属性：**

`required` - 一个数组，列出完成函数调用所必需的参数


### 调用函数
在定义好函数之后，我们需要在调用 Chat Completion API 时把它包含进去。我们通过在请求中添加 `functions` 来实现这一点。在这里，`functions=functions`。

另外，还可以选择将 `function_call` 设置为 `auto`。这意味着我们让大语言模型根据用户消息自动决定应该调用哪个函数，而不是由我们手动指定。


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

现在让我们来看一下响应，并看看它的格式：

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

你可以看到，调用的函数名称已经给出，并且从用户消息中，LLM 能够找到数据来填充函数的参数。


## 3.将函数调用集成到应用程序中

在我们测试过来自LLM的格式化响应后，现在可以将其集成到应用程序中了。

### 管理流程

要将其集成到我们的应用程序中，可以按照以下步骤操作：

首先，调用OpenAI服务，并将消息存储在名为`response_message`的变量中。


In [ ]:
response_message = response.choices[0].message

现在我们将定义一个函数，该函数将调用 Microsoft Learn API 以获取课程列表：


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



作为最佳实践，我们会先查看模型是否想要调用某个函数。之后，我们会创建一个可用的函数，并将其与被调用的函数进行匹配。

接下来，我们会获取该函数的参数，并将其映射到来自LLM的参数。

最后，我们会附加函数调用消息以及`search_courses`消息返回的值。这样，LLM就拥有了用自然语言回应用户所需的全部信息。


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## 编程挑战

做得很好！如果你想继续学习 OpenAI 的函数调用，可以尝试以下内容：https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - 为函数添加更多参数，帮助学习者找到更多课程。你可以在这里查看可用的 API 参数：
 - 创建另一个函数调用，获取学习者的更多信息，比如他们的母语
 - 当函数调用或 API 调用没有返回合适的课程时，添加错误处理



---

**免责声明**：  
本文件由 AI 翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻译。我们力求准确，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文件应被视为权威来源。对于关键信息，建议使用专业人工翻译。因使用本翻译而产生的任何误解或曲解，我们概不负责。
